In [1]:
#!pip install umap-learn
import umap
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def nearest_neighbour_distance(g):
    d = np.linalg.norm(g[:,None] - g[None],axis =-1)
    a = np.amax(d)
    for i in range(len(d)):
        d[i,i] = a
    return np.amin(d, axis = -1)

In [ ]:
class TorusOptimizer(tf.keras.Model):
    def __init__(self, l2 = 1, n_units = 3, n_waves = 2, theta = np.pi/3, **kwargs):
        '''
        Tensorflow Keras model for performing optimizing of Torus
        '''
        super().__init__(**kwargs)

        #rot = lambda x, t: np.array([[np.cos(t), -np.sin(t)], [np.sin(t), np.cos(t)]])@x
        self.l2 = l2

        # wave vectors
        #k = np.array([rot(np.array([1,0]), i*theta) for i in range(n_waves)])[None]*np.ones((n_units, 1, 1))
        #self.k = self.add_weight("k", shape = k.shape, initializer = tf.keras.initializers.Constant(k), trainable = True)
        t = np.random.uniform(0, 2*np.pi, (n_waves))
        self.theta = self.add_weight("t", shape = t.shape, initializer = tf.keras.initializers.Constant(t), trainable = True)
        # amplitudes
        w = np.random.uniform(0, 1, (n_units, n_waves))
        self.w = tf.ones(w.shape) 
        #self.w = self.add_weight("w", shape = w.shape, initializer = tf.keras.initializers.Constant(w), trainable = True)
        # phases
        phi = np.random.uniform(0, 2*np.pi, (n_units, 2))
        self.phi = self.add_weight("phi", shape = phi.shape, initializer = tf.keras.initializers.Constant(phi), trainable = True)

    @tf.function
    def grid_func(self, r):
        k = tf.stack([tf.math.cos(self.theta), tf.math.sin(self.theta)], axis = -1)
        rhat = r[:,None] - self.phi[None]
        kr = tf.reduce_sum(k[None,None]*rhat[...,None,:], axis = -1)
        g =  tf.reduce_sum(self.w[None]*tf.math.cos(kr), axis = -1)
        return g

    @tf.function
    def grid_func_jacobian(self, r):
        k = tf.stack([tf.math.cos(self.theta), tf.math.sin(self.theta)], axis = -1)
        rhat = r[:,None] - self.phi[None]
        kr = tf.reduce_sum(k[None,None]*rhat[...,None,:], axis = -1)
        Jx = -tf.reduce_sum(k[None,None,:,0]*self.w[None]*tf.math.sin(kr), axis = -1)
        Jy = -tf.reduce_sum(k[None,None,:,1]*self.w[None]*tf.math.sin(kr), axis = -1)
        J = tf.stack((Jx, Jy), axis = -1)
        return J

    @tf.function
    def grid_func_jacobian_det(self, J):
        det_squared = tf.linalg.det(tf.transpose(J, (0, 2, 1))@J)
        return det_squared

    def call(self, inputs, training = True):
      # compute Jacobian determinant
      j = self.grid_func_jacobian(inputs)
      det_j = self.grid_func_jacobian_det(j)
      # add amplitude regularization
      #l2_loss = self.l2*tf.reduce_sum(self.w**2)  
      #self.add_loss(l2_loss)
      # minimize negative/ maximize jacobian determinant
      self.add_loss(-tf.reduce_mean(det_j))
      return tf.zeros(inputs[0].shape)

In [ ]:
def fetch_model(dynamic, lr, l2 = 0.1, n_units = 3, n_waves = 3, theta = np.pi/3):
    model = TorusOptimizer(l2, n_units, n_waves, theta, dynamic = dynamic)
    model.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(lr),
                  metrics=['mae'])
    return model

model = fetch_model(False, lr = 1e-3, n_units = 3)

In [ ]:
# plot meshgrid
n_samples = 50
x = np.linspace(0, 2*np.pi, n_samples, endpoint = False)
y = np.linspace(0, 2*np.pi, n_samples, endpoint = False)

xx, yy = np.meshgrid(x, y)
u = np.stack([xx.ravel(), yy.ravel()], axis = -1).astype("float32")

# plot jacobian determinant
J = model.grid_func_jacobian(u)
det_J = model.grid_func_jacobian_det(J)
det_J_pre_train = np.reshape(det_J.numpy(), (n_samples, n_samples))

In [ ]:
# UMAP before training
gu = model.grid_func(u)
reducer = umap.UMAP(n_components = 3)
gumap = reducer.fit_transform(gu)
nn = nearest_neighbour_distance(gumap)
marker=dict(size=4,color=nn,colorscale='Viridis',opacity=1)
fig = go.Figure(data=[go.Scatter3d(x=gumap[:,0], y=gumap[:,1], z=gumap[:,2],
                                   mode='markers', marker = marker)])
fig.show()

In [ ]:
saver = tf.keras.callbacks.ModelCheckpoint(
    filepath='./',
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True) 

r = np.random.uniform(0, 2*np.pi, (50000, 2)).astype("float32") # training data :)
model.fit(r, np.zeros(r.shape), epochs = 50, batch_size = 50, callbacks = [saver])

In [ ]:
# plot jacobian determinant
fig, axs = plt.subplots(1, 2)
J = model.grid_func_jacobian(u)
det_J = model.grid_func_jacobian_det(J)
det_J = np.reshape(det_J.numpy(), (n_samples, n_samples))
# visualize Jacobian determinants before/after training
axs[0].imshow(det_J_pre_train, origin = "lower", cmap = "jet")
axs[1].imshow(det_J, origin = "lower", cmap = "jet") 
axs[0].set_title("Before Training")
axs[1].set_title("After Training")

In [ ]:
# plot grid functions 
gu = model.grid_func(u)
g = np.reshape(gu, (n_samples, n_samples, gu.shape[-1]))

for i in range(3):
  plt.imshow(g[...,i])
  plt.show()

In [ ]:
# UMAP after training
gu = model.grid_func(u)
reducer = umap.UMAP(n_components = 3)
gumap2 = reducer.fit_transform(gu)
nn = nearest_neighbour_distance(gumap)
marker=dict(size=4,color=nn,colorscale='Viridis',opacity=1)
fig = go.Figure(data=[go.Scatter3d(x=gumap2[:,0], y=gumap2[:,1], z=gumap2[:,2],
                                   mode='markers', marker = marker)])
fig.show()

In [ ]:
plt.plot(*model.phi.numpy().T, "o")

In [ ]:
# UMAP after training
gu = model.grid_func(u)
nn = nearest_neighbour_distance(gu)
marker=dict(size=4,color=nn,colorscale='Viridis',opacity=1)
fig = go.Figure(data=[go.Scatter3d(x=gu[:,0], y=gu[:,1], z=gu[:,2],
                                   mode='markers', marker = marker)])
fig.show()